In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./example-analytics-data.csv", index_col=None)

In [3]:
df.head()

,User ID,Name,Date of Birth,Session ID,Date,Page URL,Pageviews,Session Duration,Bounce Rate,Device Type,Traffic Source,Location,Conversion
0,1,John Doe,3/6/2004,S001,2024-11-25,/home,5,320,20,Mobile,Organic Search,"New York, USA",Yes
1,1,John Doe,3/6/2004,S008,2024-11-26,/product/xyz,4,200,25,Desktop,Direct,"New York, USA",No
2,2,Jane Doe,6/10/2002,S002,2024-11-25,/product/abc,3,180,33,Desktop,Direct,"London, UK",No
3,2,Jane Doe,6/10/2002,S009,2024-11-26,/checkout,7,350,14,Tablet,Paid Search,"London, UK",Yes
4,3,Timothy James,11/3/1999,S003,2024-11-26,/blog/5-tips,2,90,50,Mobile,Social Media,"Sydney, AUS",No


## Prompt

Imagine you want to send the above analytics information (that came from somewhere like your internal Google Analytics platform) to ChatGPT to be analyzed to spot trends and forecast traffic.

1. What PII is in here that needs to be protected?
2. How could you mask that PII so that this data could be passed to ChatGPT?

*Fun fact:* ChatGPT generated most of this CSV, but it forgot to enclose the `Location` field in quotes, which threw me off for a long time.